# **Ejercicios de pair programming Módulo 3 Sprint 1**
## **Regresión logística: Lección 5**
### **Decision Tree**

In [1]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score , cohen_kappa_score, roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

Hasta ahora hemos ajustado el modelo usando una Regresión Logística, pero como hemos aprendido, podemos usar el Decision Tree en este tipo de problemas. Los objetivos de este pair programming :

 - Ajustad un modelo de Decision Tree a nuestros datos.


In [2]:
df = pd.read_csv("../datos/Invistico_Airline_procesado.csv", index_col = 0)
df.head()

,age,flight_distance,departure_delay_in_minutes,gender_oe,customer_type_oe,type_of_travel_oe,class_oe,seat_comfort_oe,departure_arrival_time_convenient_oe,food_and_drink_oe,...,inflight_entertainment_oe,online_support_oe,ease_of_online_booking_oe,on_board_service_oe,leg_room_service_oe,baggage_handling_oe,checkin_service_oe,cleanliness_oe,online_boarding_oe,satisfaction
0,0.236257,0.491272,-0.386481,0,0,0,0,2,4,4,...,2,3,2,2,2,1,2,2,2,0
1,-0.359008,0.297525,-0.281414,1,0,1,1,2,3,2,...,2,5,5,3,2,3,4,3,5,0
2,-0.557430,2.030540,8.176472,0,0,0,1,4,3,3,...,4,4,2,3,3,1,4,1,4,1
3,1.889773,0.487377,-0.386481,0,0,0,1,1,4,2,...,4,4,1,1,1,0,1,1,2,0
4,-0.689712,1.161111,-0.071280,0,0,0,0,3,2,2,...,3,3,3,3,5,3,3,4,3,0


In [3]:
# Separamos las variables predictoras de la variable respuesta

X = df.drop("satisfaction", axis = 1)
y = df['satisfaction']

In [4]:
# Separamos de nuevo el train y el test para comprobar cómo de bueno es neuestro modelo y ajustamos le modelo

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

arbol = DecisionTreeClassifier(random_state =0)
 
arbol.fit(x_train, y_train)


# sacamos los hiperparámetros

max_features = np.sqrt(len(x_train.columns))
print(max_features)

print(arbol.tree_.max_depth)



# hacemos las predicciones

y_pred_test_esta = arbol.predict(x_test)
y_pred_train_esta = arbol.predict(x_train)

4.58257569495584
35


> Tendremos que pasarle a nuestro modelo en hiperparámetros una profundidad máxima de 35 y en features le pondremos como máximo hasta 5, que es el resultado de la raiz cuadrada

 - Calculad las métricas a nuestro nuevo modelo.


In [5]:
def metricas(clases_reales_test, clases_predichas_test, clases_reales_train, clases_predichas_train, modelo):
    """Esta función saca las métricas para las predicciones y los datos reales que apliremos al modelos DecisionTree
        Args:
        - Las predicciones para el train y el test y los datos reales para el train y el test, además del modelo que vamos a utilizar
        Returns: nos devuelve el dataframe con las métricas."""
    # para el test
    accuracy_test = accuracy_score(clases_reales_test, clases_predichas_test)
    precision_test = precision_score(clases_reales_test, clases_predichas_test)
    recall_test = recall_score(clases_reales_test, clases_predichas_test)
    f1_test = f1_score(clases_reales_test, clases_predichas_test)
    kappa_test = cohen_kappa_score(clases_reales_test, clases_predichas_test)

    # para el train
    accuracy_train = accuracy_score(clases_reales_train, clases_predichas_train)
    precision_train = precision_score(clases_reales_train, clases_predichas_train)
    recall_train = recall_score(clases_reales_train, clases_predichas_train)
    f1_train = f1_score(clases_reales_train, clases_predichas_train)
    kappa_train = cohen_kappa_score(clases_reales_train, clases_predichas_train)
    

    
    df = pd.DataFrame({"accuracy": [accuracy_test, accuracy_train], 
                       "precision": [precision_test, precision_train],
                       "recall": [recall_test, recall_train], 
                       "f1": [f1_test, f1_train],
                       "kapppa": [kappa_test, kappa_train],
                       "set": ["test", "train"]})
    
    df["modelo"] = modelo
    return df

In [6]:
dt_results = metricas(y_test, y_pred_test_esta, y_train, y_pred_train_esta, "Decision Tree")
dt_results

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.946454,0.944612,0.948576,0.94659,0.892908,test,Decision Tree
1,1.000000,1.000000,1.000000,1.00000,1.000000,train,Decision Tree


>Como las métricas nos indican un claro problema de overfitting, procedemos a usar GridSearchCV para buscar los mejores hiperparámetros.

In [7]:
# Creamos un diccionario con los parámetros que calculamos anteriormente.

param = {"max_depth": [5,10,15,20,25],
        "max_features": [1,2,3,4,5],
        "min_samples_split": [10, 50, 100],
        "min_samples_leaf": [10,50,100]} 

In [8]:
# Ajustamos el modelo con GridSearch pasándole el diccionario de parámetros

gs = GridSearchCV(
            estimator=DecisionTreeClassifier(random_state= 42), 
            param_grid= param,
            cv=10,
            verbose=-1)

In [9]:
gs.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_depth': [5, 10, 15, 20, 25],
                         'max_features': [1, 2, 3, 4, 5],
                         'min_samples_leaf': [10, 50, 100],
                         'min_samples_split': [10, 50, 100]},
             verbose=-1)

In [10]:
# Sacamos el mejor modelo que se ha ejecutado con GridSearch

mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeClassifier(max_depth=25, max_features=5, min_samples_leaf=10,
                       min_samples_split=10, random_state=42)

> Creamos el segundo modelo Decision Tree con los parámetros mejorados, sacados en el anterior modelo

In [11]:
arbol2 = DecisionTreeClassifier(max_depth=25, max_features=5, min_samples_leaf=10,
                       min_samples_split=10, random_state=42)
 
arbol2.fit(x_train, y_train)


# hacemos las predicciones

y_pred_test_gs= arbol2.predict(x_test)
y_pred_train_gs = arbol2.predict(x_train)

In [12]:
dt_results_gs = metricas(y_test, y_pred_test_gs, y_train, y_pred_train_gs, "Decision Tree gs")
dt_results_gs

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.93386,0.945545,0.920811,0.933014,0.867722,test,Decision Tree gs
1,0.95073,0.962340,0.938163,0.950098,0.901460,train,Decision Tree gs


 - Comparad las métricas con el modelo hecho hasta ahora. ¿Cuál es mejor?

In [13]:
# Visualizamos las métricas del modelo de regresión lineal

resultados_metricas = pd.read_csv("../datos/Invistico_Airline_metricas.csv", index_col = 0)
resultados_metricas.head()

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.844677,0.848542,0.839300,0.843896,0.689355,test,Regresión logistica
1,0.842790,0.848524,0.834521,0.841464,0.685579,train,Regresión logistica


In [14]:
# Unimos los dos modelos en un mismo dataframe

resultados_metricas = pd.concat([resultados_metricas, dt_results_gs], axis = 0)
resultados_metricas

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.844677,0.848542,0.839300,0.843896,0.689355,test,Regresión logistica
1,0.842790,0.848524,0.834521,0.841464,0.685579,train,Regresión logistica
0,0.933860,0.945545,0.920811,0.933014,0.867722,test,Decision Tree gs
1,0.950730,0.962340,0.938163,0.950098,0.901460,train,Decision Tree gs


>Hemos decidido no incluir el primer modelo de Decisión Tree (antes de ajustar los hiperparámetros) porque tenía un claro problema de overfitting.
>Vemos que el DecisionTree nos ha dado  mejores resultados que el modelo de regresión logística sin indicios de overfitting o underfitting, y que el kappa y las métricas generales tienen un valor bastante alto.

In [15]:
# Nos guardamos los resultados en un csv para poder compararlos con los siguientes que saquemos en el Random Forest

resultados_metricas.to_csv("../datos/Invistico_Airline_metricas.csv")